In [0]:
import random
import numpy as np
import matplotlib.pyplot as plt

In [0]:
def MSE(out, tgt):
    N = len(out)
    res = sum([(tgt[n]-out[n])**2 for n in range(N)]) / N
    return res

In [0]:
def display_res(genes): #Display results as input shape
  result = [int(round(element*MAX_VALUE,0)) for element in genes]
  print(result)

In [0]:
class Individual:
    def __init__(self, n_genes):
        self.genes = [random.random() for _ in range(n_genes)] #Initialization with random genes
        self.error = 0.0
            
    def set_genes(self, genes): #Replace the random genes with the recieved ones
        self.genes = genes.copy()

    def set_error(self, error):
      self.error = error

In [0]:
class Population:
    def __init__(self, population_size, n_genes):
        self.population_size = population_size
        self.n_genes = n_genes
        self.population = []
        

    def check_convergence(self, boundary_error):
      errors = [individual.error for individual in self.population]
      least_error = min(errors)
      least_error_index = errors.index(least_error)
      res = self.population[least_error_index].genes
      if least_error < boundary_error:
        return res, least_error, True
      else:  
        return res, least_error, False

    def add(self, individual): #Add the new individual to the population
        self.population.append(individual)
        
    def rand_individuals(self): #Fill the population with random individuals
        self.population = [Individual(self.n_genes) for _ in range(self.population_size)]
        
    def eval_population(self, tgt_func, tgt_value):
      for individual in self.population:
        error = tgt_func(individual.genes, tgt_value)
        individual.set_error(error)
                    
    def selection(self, num_individ=5): #турнирная селекция
        self.selected = []
        for individual in self.population:
          rivals = [random.choice(self.population) for _ in range(num_individ)] #select rivals from population
          rivals_error = [individual.error for individual in rivals]
          least_error = min(rivals_error)
          best_individ_index = rivals_error.index(least_error)
          best_individ = rivals[best_individ_index]
          self.selected.append(best_individ)
        
    def crossbreeding(self, param, probability): #арифметический кроссинговер  
        self.temp_population = Population(self.population_size, self.n_genes)
        
        for _ in range(self.population_size//2): #т.к. в результате скрещивания появляется 2 потомка
          parent_1 = self.selected[random.choice(np.arange(self.population_size))] 
          parent_2 = self.selected[random.choice(np.arange(self.population_size))] 
          child_1 = Individual(self.n_genes)
          child_2 = Individual(self.n_genes)
          if random.random() < probability:
            child_1.set_genes(self.child_1_genes(parent_1, parent_2, param))
            child_2.set_genes(self.child_2_genes(parent_1, parent_2, param))
          self.temp_population.add(child_1)
          self.temp_population.add(child_2)
    
    def child_1_genes(self, parent_1, parent_2, param):
      child_genes = []
      for num in range(self.n_genes):
        gene = param * parent_1.genes[num] + (1 - param) * parent_2.genes[num]
        child_genes.append(gene)
      return child_genes
    
    def child_2_genes(self, parent_1, parent_2, param):
      child_genes = []
      for num in range(self.n_genes):
        gene = param * parent_2.genes[num] + (1 - param) * parent_1.genes[num]
        child_genes.append(gene)
      return child_genes 

    def new_population(self, generation_bridge):
      for num in range(self.population_size):
        if random.random() < generation_bridge:
          self.temp_population.population[num].genes = self.selected[num].genes
      return self.temp_population
    
    def mutate(self, epsilon):
        probability = 1 / self.n_genes
        for individual in self.population:
            for gene in individual.genes:
                if random.random() < probability:
                    rnd = random.choice([-1,1,-1,1,-1,1,-1,1]) * random.random() * epsilon #random distribution
                    gene = gene + rnd

In [0]:
random.seed(0)
target = [96,96,159]
MAX_VALUE = 255
norm_target = [element/MAX_VALUE for element in target]

BOUNDARY_ERROR = 0.000001
PROBABILTY = 0.9 #Chance of crossbreeding
PARAM = 0.5 #Lambda in crossbreeding
EPSILON = 0.1 #Mutation value
GENERATION_BRIDGE = 0.4 #Chance for old generations to remain in population
POPULATION_SIZE = 1000
N_GENES = len(norm_target)
EPOCHES = 10000

PROGRESS_BAR_UNIT = EPOCHES / 10

populations = []

In [0]:
#The first steps
populations.append(Population(POPULATION_SIZE,N_GENES)) #The first generation
populations[0].rand_individuals() #Initialize the first generation

done = False
#The main cycle
for epoch in range(0, EPOCHES):
   populations[epoch].eval_population(MSE, norm_target) #Evaluate the population
   #Check if converged
   res, error, done = populations[epoch].check_convergence(BOUNDARY_ERROR)
   if done == True:
     print('Done in ', epoch, ' epoches; ', 'error: ', error)
     display_res(res)
     break

   populations[epoch].selection()
   populations[epoch].crossbreeding(PARAM, PROBABILTY)
   populations[epoch].mutate(EPSILON)
   populations.append(populations[epoch].new_population(GENERATION_BRIDGE))

   if epoch % PROGRESS_BAR_UNIT == 0: print(epoch/EPOCHES*100, '%')

0.0 %
Done in  5  epoches;  error:  5.227319721241434e-07
[96, 96, 159]


In [0]:
if done == False:
  print('100 %')
  final_individuals = populations[-1] #The last element
  final_errors = [individual.error for individual in final_individuals.population]
  least_error = min(final_errors)
  best_individual_index = final_errors.index(least_error)
  best_individual = final_individuals.population[best_individual_index]
  result = [int(round(element*MAX_VALUE,0)) for element in best_individual.genes]

  print('target: ', target)
  print('result: ', result)
  print('Error: ', least_error)